Set MLflow tracker

In [18]:
import mlflow
# Provide uri and connect to your tracking server
mlflow.set_tracking_uri('http://127.0.0.1:5000/')

In [26]:
COUNTRY = "US"

model_name = "Logistic_Regression"
model_version = 1
model_description = "This model forecasts the oil production based on the previous data."
version_description = "mlflow model version"

In [27]:
model_name = f"{COUNTRY}_{model_name}"

'US_Logistic_Regression'

### MLflow Model Registry 

In [20]:
import time
from mlflow.tracking.client import MlflowClient
from mlflow.entities.model_registry.model_version_status import ModelVersionStatus
 
def wait_until_ready(model_name, model_version):
    client = MlflowClient()
    for _ in range(10):
        model_version_details = client.get_model_version(
        name=model_name,
        version=model_version,
        )
        status = ModelVersionStatus.from_string(model_version_details.status)
        print("Model status: %s" % ModelVersionStatus.to_string(status))
        if status == ModelVersionStatus.READY:
            break
        time.sleep(1)

In [21]:
wait_until_ready(model_name, model_version)

Model status: READY


In [22]:
from mlflow.tracking.client import MlflowClient
 
client = MlflowClient()
client.update_registered_model(
  name=model_name,
  description=model_description
)

<RegisteredModel: creation_timestamp=1677360384134, description='This model forecasts the oil production based on the previous data.', last_updated_timestamp=1677377431702, latest_versions=[<ModelVersion: creation_timestamp=1677360384202, current_stage='Production', description='mlflow model version', last_updated_timestamp=1677377336232, name='AU_Logistic_Regression', run_id='cd134af79de24ac18da83aa1951dc7df', run_link='', source='./mlflow/cd134af79de24ac18da83aa1951dc7df/artifacts/Logistic_Regression', status='READY', status_message='', tags={}, user_id='', version='1'>], name='AU_Logistic_Regression', tags={}>

In [23]:
client.update_model_version(
  name=model_name,
  version=model_version,
  description=version_description
)

<ModelVersion: creation_timestamp=1677360384202, current_stage='Production', description='mlflow model version', last_updated_timestamp=1677377432805, name='AU_Logistic_Regression', run_id='cd134af79de24ac18da83aa1951dc7df', run_link='', source='./mlflow/cd134af79de24ac18da83aa1951dc7df/artifacts/Logistic_Regression', status='READY', status_message='', tags={}, user_id='', version='1'>

In [24]:
client.transition_model_version_stage(
  name=model_name,
  version=model_version,
  stage='Production',
)

<ModelVersion: creation_timestamp=1677360384202, current_stage='Production', description='mlflow model version', last_updated_timestamp=1677377433939, name='AU_Logistic_Regression', run_id='cd134af79de24ac18da83aa1951dc7df', run_link='', source='./mlflow/cd134af79de24ac18da83aa1951dc7df/artifacts/Logistic_Regression', status='READY', status_message='', tags={}, user_id='', version='1'>

In [25]:
model_version_details = client.get_model_version(
  name=model_name,
  version=model_version,
)
print("The current model stage is: '{stage}'".format(stage=model_version_details.current_stage))

The current model stage is: 'Production'


In [17]:
latest_version_info = client.get_latest_versions(model_name, stages=["Production"])
latest_production_version = latest_version_info[0].version
print("The latest production version of the model '%s' is '%s'." % (model_name, latest_production_version))

The latest production version of the model 'AU_Logistic_Regression' is '1'.
